# 머신러닝
       
       
3주 1강: Numpy Basics
<br/><br/>

숭실대학교<br/>
AI융합학부<br/>
윤진혁<br/>

## Why NumPy

Python에서 수치 데이터를 조작하는 것은 몇가지 방법이 있습니다.

기본적인 Python의 기능과 자료형으로도 충분히 데이터를 조작할 있습니다. 

하지만 Python의 기본 기능들은 꽤나 느린 편입니다. 

그래서속도와 편의성 측면에서 대부분 NumPy를 사용하게 됩니다.

이번 주에는 NumPy의 Array(배열)을 사용해서 데이터와 하위 배열을 조회, 분할하고 형태를 변경하는 등을 배워봅니다.

사실 뒤에 배울 Pandas도 내부적으로는 NumPy array를 사용해서 구현되어 있습니다.

오늘 강의는 Python Data Science Handbook by Jake VanderPlas 을 참조했습니다.

In [1]:
import numpy
numpy.__version__

'2.1.0'

## 효율적인 데이터 분석에 관하여

데이터 분석 처리를 위해서는 실제로 데이터가 Python 내부에서 어떻게 저장되고 처리되는지에 대해서 배우는 것이 필요합니다. 

이번 강의를 통해서 Python이 데이터를 다루는 방법과, Numpy가 이 방식을 어떻게 개선해서 효율성을 높이는지에 대해서 이야기 해 보겠습니다. 

Python은 때때로 사용이 쉬운 것으로 알려져 있습니다. 그 이유중 하나는 Python의 변수는 그 자료형이 능동적으로 변화할 수 있기 때문입니다.



예를 들어서 Python은 아래와 같은 형태의 코드를 입력해도 잘 작동합니다.

```python
# Python code
x = 4
x = "four"
```

자료형을 명시적으로 선언하지도 않았고, int가 assign된 곳에서 string을 입력해도 잘 작동하지요

```C
/* C code */
int x = 4;
x = "four";  // FAILS
```

이렇게 작동이 가능한 "이유"를 아는 것이 효율적인 python 프로그래밍을 위해서는 매우 중요합니다.
즉, C의 integer와 Python의 integer가 메모리를 어떻게 다르게 쓰고 있는지를 아는 것이 중요합니다.

## A Python Integer Is More Than Just an Integer

Python의 기본 인터프리터는 C로 작성되어 있습니다.
즉 다시 말해서, python의 자료형은 C 자료형을 Python 기능에 맞추어서 만들어놓은 Wrapper Class라는 뜻입니다. 
예를 들어서, python의 interger는 사실 아래와 같은 자료구조입니다. 

```C
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```

즉, Python의 interger는 아래의 4가지로 구성되어 있습니다:

- ``ob_refcnt``, Python이 Memory allocation과 deallocation에 쓰는 reference counta reference count.
- ``ob_type``, 변수형
- ``ob_size``, 변수의 크기
- ``ob_digit``, long으로 되어 있는 실제 변수값.

즉, 아래의 그림처럼 C의 자료형을 wrapping한 형태입니다.

![Integer Memory Layout](Figs/cint_vs_pyint.png)

``PyObject_HEAD``는 자료 실제 값을 제외한 나머지 3가지를 의미합니다.
즉, C의 integer는 자료의 값이 들어있는 memory를 바로 조회하게 되어있습니다. 
python의 integer는 object information이 들어있는 자료형을 가리키는 pointer에 가깝게 구성되어 있습니다.
즉, integer에 string을 assingn하는 것은 내부적으로는 새 메모리를 할당하고, string 자료형을 가리키게 포인터를 바꿔주고, 기존의 interger 메모리를 할당해제주는 과정을 거치게 됩니다. 즉, operation의 수가 많아지고, 비용이 증가할 수 밖에 없습니다.

## A Python List Is More Than Just a List

단순 자료형이 아니라 여러개의 오브젝트를 동시에 가지고 있는 복합자료형에 대해서 생각해봅시다. 
일단 리스트를 먼저 생각해 봅시다.

In [14]:
L = list(range(10))
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [15]:
type(L[0])

int

리스트는 String으로도 만들 수 있지요.

In [4]:
L2 = [str(c) for c in L]
L2

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [6]:
type(L2[0])

str

지난 시간에 배웠듯 python은 한 리스트 안에 다양한 자료형을 넣을 수 있습니다.

In [6]:
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]

[bool, str, float, int]

이러한 모든 것이 가능한 이유는 사실 python의 list는 c의 array와는 다르게, 메모리의 연속 공간을 차지하게 구성되어 있지 않기 때문입니다.
그리고 각 element object가 위의 integer 예처럼 PyObject_HEAD를 별도로 가지고 있는 구조입니다.

즉. 이 모든 것이 일종의 "비용"이 됩니다.

그에 비해서 NumPy의 Array는 조금 더 C의 Array에 가깝게 되어 있습니다.
Numpy Array 자체의 PyObject_HEAD는 있으나, 각 element는 C의 자료형처럼 연속된 메모리 공간을 차지합니다:

![Array Memory Layout](Figs/array_vs_list.png)

## Fixed-Type Arrays in Python

Python에는 List말고도 array라는 모듈이 있습니다. 이 Array의 모듈은 조금 더 C에 가깝게 변수형이 고정되어 있습니다 (Python 3.3부터 존재합니다):

In [7]:
import array
L = list(range(10))
A = array.array('i', L)
A

array('i', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

여기에서 "i"는 이 array가 integer type임을 의미합니다. 
이 array와 유사하지만, 이 array를 다루는 operation들은 그다지 많지 않습니다.

Numpy의 ``ndarray``는 이러한 array와 유사한 object이지만, 훨씬 더 유연하게 쓸 수 있습니다.

아래처럼 보통 np라는 이름으로 numpy를 import 하곤 합니다.

In [1]:
import numpy as np

## Creating Arrays from Python Lists

numpy array는 python list로부터 생성 가능합니다.

In [9]:
# integer array:
np.array([1, 4, 2, 5, 3])

array([1, 4, 2, 5, 3])

numpy array는 꼭 같은 타입의 변수들로 만들어야 합니다.

만약 변수들의 타입이 다른 list라면 typecast가 가능한 경우라면 typecast로 한 변수로 맞추어 줍니다.

예를 들어 아래의 경우 integer들이 float으로 upcast됩니다.

In [10]:
np.array([3.14, 4, 2, 3])

array([3.14, 4.  , 2.  , 3.  ])

만약 정해진 type의 array를 만들고 싶다면 ``dtype``을 지정해주면 됩니다.

In [11]:
np.array([1, 2, 3, 4], dtype='float32')

array([1., 2., 3., 4.], dtype=float32)

또한 numpy의 array는 다차원 array를 기본으로 합니다.

list of list를 array로 만들면 다차원 array가 됩니다.

In [2]:
# nested lists 는 다차원 배열이 됩니다.
np.array([range(i, i + 3) for i in [2, 4, 6]])

array([[2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

In [15]:
# 아래와 같이 만들면 list의 1d array가 되는 점에 주의해 주세요.
np.array([[1, 2],[3],[4, 5]], dtype=object)

array([list([1, 2]), list([3]), list([4, 5])], dtype=object)

## Creating Arrays from Scratch

Numpy에는 Array를 만들 수 있는 다양한 방법이 있습니다. 

큰 Array를 만들떄는 이 방법들이 효율적일 수 있습니다.

In [16]:
# 0으로 채워진 길이10 integer array를 만들어 봅시다.
np.zeros(10, dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [17]:
# 1로 채워진 3x5 floating-point array를 만들어 봅시다.
np.ones((3, 5), dtype=float)

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [18]:
# 3.14로 채워진 3x5 array를 생성해 봅시다.
np.full((3, 5), 3.14)

array([[3.14, 3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14, 3.14]])

In [19]:
# 어떤 linear sequence로 구성된 array를 만들어 봅시다.
# 0부터 시작해서 20에서 끝나고, 각 성분은 2씩 차이가 나게 만들어 봅시다. 
# python의 기본 range()함수와 비슷하지요?
np.arange(0, 20, 2) 

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [20]:
# 0부터 1 사이에 균일하게 나누어진 5개의 값으로 array를 생성해 봅시다.
np.linspace(0, 1, 5)

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [21]:
# 0부터 1까지 uniform distribution의 난수로 만든
# 3x3 array를 만들어 봅시다. 
np.random.random((3, 3))

array([[0.10832372, 0.81751657, 0.70908458],
       [0.23527649, 0.36665772, 0.42522818],
       [0.48584054, 0.1016245 , 0.88154436]])

In [22]:
# gaussian distribution의 난수로 만든
# 3x3 array를 만들어 봅시다. 
# mean 0 standard deviation 1을 가정합니다.
np.random.normal(0, 1, (3, 3))

array([[-0.84834753,  0.04326747,  0.56397966],
       [ 1.73801741,  0.19187791, -0.98462648],
       [-0.85094286, -0.3493658 ,  1.06668236]])

In [23]:
# [0,10) 의 random integer로 만든 3*3 행렬을 만들어봅시다.
np.random.randint(0, 10, (3, 3))

array([[4, 5, 3],
       [6, 1, 1],
       [2, 5, 6]])

In [24]:
# 단위행렬을 만들어 봅시다.
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [25]:
# 비어있는 size 3 행렬을 만들어 봅시다.
# 주의: 여기 채워져있는 값 들은 초기화가 되어있지 않은 값들입니다.
np.empty(3) 

array([1., 1., 1.])

## NumPy Standard Data Types

Numpy array는 기본적으로 한가지 종류의 데이터로 구성되어 있어야 합니다.

즉, 어떤 Data Type들이 있는지를 아는 것이 중요하고, 또 각 Data type의 한계를 이해하는 것도 중요합니다. 

C에 익숙한 분들이면 아래의 형태의 

Array를 만들 때 string을 통해서 datatype을 정해줄 수 있습니다.

```python
np.zeros(10, dtype='int16')
```

혹은 Numpy data object type을 직접 적어주어도 됩니다.

```python
np.zeros(10, dtype=np.int16)
```

| Data type	    | Description |
|---------------|-------------|
| ``bool_``     | Boolean (True or False) stored as a byte |
| ``int_``      | Default integer type (same as C ``long``; normally either ``int64`` or ``int32``)| 
| ``intc``      | Identical to C ``int`` (normally ``int32`` or ``int64``)| 
| ``intp``      | Integer used for indexing (same as C ``ssize_t``; normally either ``int32`` or ``int64``)| 
| ``int8``      | Byte (-128 to 127)| 
| ``int16``     | Integer (-32768 to 32767)|
| ``int32``     | Integer (-2147483648 to 2147483647)|
| ``int64``     | Integer (-9223372036854775808 to 9223372036854775807)| 
| ``uint8``     | Unsigned integer (0 to 255)| 
| ``uint16``    | Unsigned integer (0 to 65535)| 
| ``uint32``    | Unsigned integer (0 to 4294967295)| 
| ``uint64``    | Unsigned integer (0 to 18446744073709551615)| 
| ``float_``    | Shorthand for ``float64``.| 
| ``float16``   | Half precision float: sign bit, 5 bits exponent, 10 bits mantissa| 
| ``float32``   | Single precision float: sign bit, 8 bits exponent, 23 bits mantissa| 
| ``float64``   | Double precision float: sign bit, 11 bits exponent, 52 bits mantissa| 
| ``complex_``  | Shorthand for ``complex128``.| 
| ``complex64`` | Complex number, represented by two 32-bit floats| 
| ``complex128``| Complex number, represented by two 64-bit floats| 

## 앞으로 배울 것들
일단 기본 배열의 조작을 위한 아래의 것 들을 배울 것입니다.

- 배열의 속성: 배열의 크기, 모양, 데이터타입 등
- 배열의 인덱싱: 개별 배열의 요소를 조회하고 바꾸는 법
- 배열의 분할 (slicing): 큰 배열에서 작은 하위 배열을 가져오는 방법
- 배열의 모양 변경 (reshape): 배열의 형태를 변경하는 방법.
- 배열의 결합 (join) 과 분할 (split): 여러 배열을 하나로 합치거나 하나의 배열을 여러개로 나누는 법

## Array의 속성
NumPy array의 종류와 속성값을 먼저 살펴봅시다. 
임의의 값으로 채워진 Random Array를 만들어서, Array에 대한 테스트를 해 보겠습니다. 

In [3]:
import numpy as np
np.random.seed(0)  # seed for reproducibility

x1 = np.random.randint(10, size=6)  # One-dimensional array
x2 = np.random.randint(10, size=(3, 4))  # Two-dimensional array
x3 = np.random.randint(10, size=(3, 4, 5))  # Three-dimensional array

각각의 array는 ``ndim``, ``shape``, ``size`` 라는 attribute들을 가지고 있습니다. ``ndim``은 array의 차원, ``shape``는 각 dimension의 크기, ``size``는 전체 array의 크기를 나타냅니다. 

In [ ]:
print("x3 ndim: ", x3.ndim)  # array의 차원
print("x3 shape:", x3.shape)  # 각 dimension의 크기
print("x3 size: ", x3.size)  # 전체 array의 크기

x3 ndim:  3
x3 shape: (3, 4, 5)
x3 size:  60


``dtype``이라는 Attribute도 중요한 역할을 합니다. Array의 data type을 나타내 줍니다. 

In [28]:
print("dtype:", x3.dtype)

dtype: int64


``itemsize``라는 속성값도 중요한 역할을 합니다. 이 값은 배열 성분들 한 개의 데이터 사이즈를 의미합니다.

비슷하게 ``nbytes``라는 속성값은 이 배열의 총 용량을 의미합니다.

In [29]:
print("itemsize:", x3.itemsize, "bytes")
print("nbytes:", x3.nbytes, "bytes")

itemsize: 8 bytes
nbytes: 480 bytes


## Array Indexing: Single Elements

Python의 list index에 익숙하다면, Numpy의 Index도 사실 비슷하게 느낄 것입니다.

In [30]:
x1

array([5, 0, 3, 3, 7, 9])

In [31]:
# index는 0부터 시작합니다.
print(x1[0])
print(x1[4])

5
7


In [32]:
# Negative index도 사용 가능합니다.
print(x1[-1])
print(x1[-2])

9
7


다차원 배열은 comma separated index로 조회가 가능합니다. 

Note: Python의 2차원 배열은 [a][b] 형태로 사용했었던 기억이 나시죠?

In [33]:
x2

array([[3, 5, 2, 4],
       [7, 6, 8, 8],
       [1, 6, 7, 7]])

In [34]:
x2[0,0]

np.int64(3)

In [35]:
x2[2, 0]

np.int64(1)

In [36]:
x2[2, -1]

np.int64(7)

In [37]:
x2[2, -1]

np.int64(7)

위의 방법중 어떤 방법으로 index를 불러와도 그 값을 변경할 수 있습니다.

In [38]:
x2[0, 0] = 12
x2

array([[12,  5,  2,  4],
       [ 7,  6,  8,  8],
       [ 1,  6,  7,  7]])

다시 한번 말씀드리지만 Numpy의 Array는 Type이 고정적입니다.

만약 int array에 float을 넣으면 truncated되어서 소수점 아래는 모두 버려지게 됩니다.
만약 이런 실수를 한다면 수치계산에서는 치명적이겠죠?

In [39]:
x1[0] = 3.14159  # this will be truncated!
x1

array([3, 0, 3, 3, 7, 9])

## Array Slicing: Accessing Subarrays

Array의 slicing도 ``[]`` 을 통해서 가능합니다.
slicing을 할 때는 ``[:]``의 형태를 취합니다.

Python의 List Slicing과 매우 유사한 방식이므로, Python의 List에 익숙하시다면 어려움은 없으실 겁니다.

``` python
x[start:stop:step]
```

참고로 저 3가지들 중 지정되지 않은 값은 아래와 같은 기본 값을 가집니다.
* ``start=0``
* ``stop=``*``size of dimension``*
* ``step=1``

### One-dimensional subarrays

In [40]:
x = np.arange(10)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [41]:
x[:5]  # first five elements

array([0, 1, 2, 3, 4])

In [42]:
x[5:]  # elements after index 5

array([5, 6, 7, 8, 9])

In [43]:
x[4:7]  # middle sub-array

array([4, 5, 6])

In [44]:
x[::2]  # every other element

array([0, 2, 4, 6, 8])

In [45]:
x[1::2]  # every other element, starting at index 1

array([1, 3, 5, 7, 9])

``step``이 음수로 주어지면, start와 stop이 뒤집힌 듯한 효과를 내 줍니다. 

만약 array를 뒤집고 싶다면 별도의 함수를 쓰지 않고 이 방식을 쓰는 것도 좋습니다.

In [46]:
x[::-1]  # all elements, reversed

array([9, 8, 7, 6, 5, 4, 3, 2, 1, 0])

In [47]:
x[5::-2]  # reversed every other from index 5

array([5, 3, 1])

### Multi-dimensional subarrays

다차원 Array도 유사하게 쓸 수 있습니다. 다차원 Array는 comma를 통해서 slicing을 해 주면 됩니다.

In [48]:
x2

array([[12,  5,  2,  4],
       [ 7,  6,  8,  8],
       [ 1,  6,  7,  7]])

In [49]:
x2[:2] # two rows

array([[12,  5,  2,  4],
       [ 7,  6,  8,  8]])

In [50]:
x2[:,:3] # three columns

array([[12,  5,  2],
       [ 7,  6,  8],
       [ 1,  6,  7]])

In [51]:
x2[:2, :3]  # two rows, three columns

array([[12,  5,  2],
       [ 7,  6,  8]])

In [52]:
x2[:, ::2]  # all rows, every other column

array([[12,  2],
       [ 7,  8],
       [ 1,  7]])

-1 을 사용하면 위의 1darray와 유사하게 뒤집는 것도 가능합니다.

In [53]:
x2[::-1, ::-1]

array([[ 7,  7,  6,  1],
       [ 8,  8,  6,  7],
       [ 4,  2,  5, 12]])

#### Accessing array rows and columns

한개의 Row나 한개의 Comlumn을 조회하는 것은 매우 자주 사용됩니다.

column이나 row의 조회는 빈 slice를 이용하면 됩니다 

In [54]:
print(x2[:, 0])  # first column of x2

[12  7  1]


In [55]:
print(x2[0, :])  # first row of x2

[12  5  2  4]


row를 조회할 때는 empty slice는 생략해도 됩니다.

In [56]:
print(x2[0])  # equivalent to x2[0, :]

[12  5  2  4]


### Subarrays as no-copy views

Array의 slice는 실제로 변수의 copy를 return해 주지 않고 그 변수의 조회를 해 주는 것에 가깝습니다.

Python의 list slice는 복사본을 return해 줍니다. 그래서 메모리 사용 등에 더 효율적입니다.

In [57]:
print(x2)

[[12  5  2  4]
 [ 7  6  8  8]
 [ 1  6  7  7]]


여기서 $2 \times 2$ subarray 를 추출합니다.

In [58]:
x2_sub = x2[:2, :2]
print(x2_sub)

[[12  5]
 [ 7  6]]


이 subarray의 변수를 바꿉니다. 

In [59]:
x2_sub[0, 0] = 99
print(x2_sub)

[[99  5]
 [ 7  6]]


In [60]:
print(x2)

[[99  5  2  4]
 [ 7  6  8  8]
 [ 1  6  7  7]]


즉, subarray 변수를 바꾸어도 실제 원래 array가 변화합니다.

이게 유용할 수도, 혼란을 가져올 수도 있습니다. 하지만 큰 데이터를 다루는 때에 작은 부분을 고치기 위해서 꽤나 유용합니다.

### Creating copies of arrays
그렇다면, "복사"를 하기 위해선 어떻게 해야할까요?

python의 ``copy`` 모듈을 써도
됩니다만. numpy에는 이미 ``copy()``라는 internal method가 있습니다. 

In [61]:
x2_sub_copy = x2[:2, :2].copy()
print(x2_sub_copy)

[[99  5]
 [ 7  6]]


이제는 이 array를 바꾸어도 원 array가 바뀌지 않습니다.

In [62]:
x2_sub_copy[0, 0] = 42
print(x2_sub_copy)

[[42  5]
 [ 7  6]]


In [63]:
print(x2)

[[99  5  2  4]
 [ 7  6  8  8]
 [ 1  6  7  7]]


## Reshaping of Arrays
Array의 Reshape도 꽤나 자주 사용하는 기능입니다.

예를 들어서 크기 9의 1d array를 $3 \times 3$ array로 바꾸는 방법은 아래와 같습니다.

In [64]:
grid = np.arange(1, 10).reshape((3, 3))
print(grid)

[[1 2 3]
 [4 5 6]
 [7 8 9]]


다만 이렇게 작동하려면 reshape 후의 array 사이즈가 원 array와 동일해야 합니다. 즉, 총 element의 수가 같게 유지되어야 합니다. 
``reshape`` method 또한 보통 단순한 조회를 해 줍니다만, 항상 그렇지는 않습니다.

가장 흔하게 하는 일 중 하나는 1darray를 2darray의 row나 column matrix로 바꾸는 것입니다. 
``reshape`` 로도 가능하고, ``newaxis``라는 것으로도 가능합니다.

In [65]:
x = np.array([1, 2, 3])

# row vector via reshape
x.reshape((1, 3))

array([[1, 2, 3]])

In [66]:
# row vector via newaxis
x[np.newaxis, :]

array([[1, 2, 3]])

In [67]:
# column vector via reshape
x.reshape((3, 1))

array([[1],
       [2],
       [3]])

In [68]:
# column vector via newaxis
x[:, np.newaxis]

array([[1],
       [2],
       [3]])

## Array Concatenation and Splitting

지금까지는 한 개의 Array를 다루는 법을 배웠다면, 지금부터는 여러 Array를 한 개로 합치거나 하나의 Array를 나누거나 하는 방법을 알아보겠습니다. 

### Concatenation of arrays

Array를 합치는 것은 ``np.concatenate``, ``np.vstack``, ``np.hstack``의 3가지 방법을 가장 많이 씁니다.
``np.concatenate`` 는 tuple이나 list 형태로 여러 array를 받아서 합쳐줍니다.

In [7]:
x = np.array([1, 2, 3])
y = np.array([3, 2, 1])
np.concatenate([x, y])

array([1, 2, 3, 3, 2, 1])

2개 이상의 Array도 한 번에 합칠 수 있습니다.

In [70]:
z = [99, 99, 99]
print(np.concatenate([x, y, z]))

[ 1  2  3  3  2  1 99 99 99]


2darray도 concantenate로 합칠 수 있습니다. 아래의 2 방법이 가능합니다.

In [8]:
grid = np.array([[1, 2, 3],
                 [4, 5, 6]])

In [9]:
# concatenate along the first axis
np.concatenate([grid, grid])

array([[1, 2, 3],
       [4, 5, 6],
       [1, 2, 3],
       [4, 5, 6]])

In [73]:
# concatenate along the second axis (zero-indexed)
np.concatenate([grid, grid], axis=1)

array([[1, 2, 3, 1, 2, 3],
       [4, 5, 6, 4, 5, 6]])

서로 다른 dimension의 array를 합치려면 vstack (vertical stack)이나 hstack(horizontal stack)을 씁니다. 

In [74]:
x = np.array([1, 2, 3])
grid = np.array([[9, 8, 7],
                 [6, 5, 4]])

# vertically stack the arrays
np.vstack([x, grid])

array([[1, 2, 3],
       [9, 8, 7],
       [6, 5, 4]])

In [75]:
# horizontally stack the arrays
y = np.array([[99],
              [99]])
np.hstack([grid, y])

array([[ 9,  8,  7, 99],
       [ 6,  5,  4, 99]])

``np.dstack``이라는 함수도 있습니다. 이것은 3번째 축에서 stack을 하는 것과 같습니다.

In [10]:
a = np.array((1,2,3))
b = np.array((2,3,4))
c = np.dstack((a,b))
print(c)
print(c.shape)

[[[1 2]
  [2 3]
  [3 4]]]
(1, 3, 2)


### Splitting of arrays

이제는 합쳐봤으니 나누는 일을 해 보겠습니다. ``np.split``, ``np.hsplit``, ``np.vsplit`` 세 가지를 사용합니다. 나누고 싶은 위치의 list를 입력하면 됩니다. 

In [11]:
x = [1, 2, 3, 99, 99, 3, 2, 1]
x1, x2, x3 = np.split(x, [3, 5])
print(x1, x2, x3)

[1 2 3] [99 99] [3 2 1]


``np.hsplit`` 과 ``np.vsplit`` 도 유사합니다.

In [78]:
grid = np.arange(16).reshape((4, 4))
grid

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [79]:
upper, lower = np.vsplit(grid, [2])
print(upper)
print(lower)

[[0 1 2 3]
 [4 5 6 7]]
[[ 8  9 10 11]
 [12 13 14 15]]


In [80]:
left, right = np.hsplit(grid, [2])
print(left)
print(right)

[[ 0  1]
 [ 4  5]
 [ 8  9]
 [12 13]]
[[ 2  3]
 [ 6  7]
 [10 11]
 [14 15]]


``np.dstack``처럼, 3번째 축으로 나누어주는 ``np.dsplit``함수도 있습니다.

# Numpy Array를 사용한 수치계산: Universal Functions

지금까지는 Array 자체에 대해서 공부해 봤습니다. 지금부터는 Numpy의 Array를 사용한 계산이 어떻게 이루어지는지, 그리고 어떻게 효율적으로 사용하는지에 대해서 배우도록 하겠습니다. 

Numpy Array를 사용한 계산은 어떻게 구현하느냐에 따라 매우 빠를수도, 매우 느릴수도 있습니다. 
빠르게 만드는 핵심은 loop의 사용을 줄이고 계산을 벡터화 하는 것입니다. NumPy의 *universal functions* (ufuncs) 은 이러한 벡터화를 지원합니다. 

## The Slowness of Loops

Python의 기본 인터프리터 구현인 CPython은 "대단히" 느립니다. 

언어의 유연성과 인터프리트 형태에서 따라오는 일종의 부작용인데요, 그래서 최대한 머신의 성능을 끌어다 C나 포트란등에 비해서 매우 느립니다. 데이터가 커 지면 치명적이지요.



몇몇가지 프로젝트가 이런 것들을 개선하기 위해서 작업중입니다. 예를 들어서 just-in-time을 도입해서 성능을 개선한 [PyPy](http://pypy.org/), Python code를 컴파일된 C코드로 바꾸어 주는 [Cython](http://cython.org), 그리고 일부 python의 코드를 LLVM 바이트코드로 바꾸어서 속도를 올리는 [Numba](http://numba.pydata.org/) 등이 있습니다. 각각 장단점이 있긴 합니다만, 100% CPython과 호환성을 보장해주지 않습니다. 예를 들어서 지금 배우는 numpy의 경우 pypy에서 꽤 오랜 기간 제대로 작동하지 않았고, 지금도 매우 느리게 작동합니다. 

Python의 loop는 정말 느립니다. 예를 들어서 리스트 element의 역수를 계산하는 아래의 계산을 한다고 해 보지요.

In [81]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output  
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

이 방식은 가장 이해하기 편하고, C나 Java등 다른 언어부터 배운 사람들에게 매우 편하게 다가올 것입니다. 

하지만 그 속도가 놀라울 정도로 느립니다. 

``%timeit`` magic operator를 사용하면 현재 cell의 loop당 실행 시간을 알 수 있습니다. 

In [82]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

1.26 s ± 17.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


100만개정도의 성분을 가진 list라면 몇초에서 몇 분 정도가 걸립니다. 
1000만개면 더 느릴 것이고, 1억개면 더 느리겠죠. 

이 과목은 "빅데이터"를 다루는 것을 가정합니다. 예를 들어 수십억개 데이터도 이제는 딱히 큰 데이터는 아닙니다. 

그렇다면 이 것을 numpy를 통해 어떻게 개선할 수 있을까요?


## Introducing UFuncs

Numpy는 *벡터화*된 연산을 통해 이를 개선합니다. Vector등의 선형대수 계산은 OpenBLAS, intel MKL같은 BLAS(Basic Linear Algebra Subprograms)라이브러리를 통해 매우 빠르게 최적화 되어 있습니다. 

현재의 numpy가 사용하는 BLAS 엔진을 한번 체크해봅시다.

In [83]:
np.show_config()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /home/jhyun/anaconda3/envs/ml2024/include
    lib directory: /home/jhyun/anaconda3/envs/ml2024/lib
    name: blas
    openblas configuration: unknown
    pc file directory: /home/jhyun/anaconda3/envs/ml2024/lib/pkgconfig
    version: 3.9.0
  lapack:
    detection method: pkgconfig
    found: true
    include directory: /home/jhyun/anaconda3/envs/ml2024/include
    lib directory: /home/jhyun/anaconda3/envs/ml2024/lib
    name: lapack
    openblas configuration: unknown
    pc file directory: /home/jhyun/anaconda3/envs/ml2024/lib/pkgconfig
    version: 3.9.0
Compilers:
  c:
    args: -march=nocona, -mtune=haswell, -ftree-vectorize, -fPIC, -fstack-protector-strong,
      -fno-plt, -O2, -ffunction-sections, -pipe, -isystem, /home/jhyun/anaconda3/envs/ml2024/include,
      -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/numpy_1724034842610/work=/usr/local/src/conda/numpy-2.1.0,
 

MKL, OPENBLAS등의 Basic library를 쓰는 것을 볼 수 있습니다. 

이제 아래의 두 식을 비교해보죠

원래의 loop식과, numpy의 vectorized operator인 ``/`` 를 비교해 봅시다. 결과적으로는 같은 결과 출력합니다.

In [84]:
print(compute_reciprocals(values))
print(1.0 / values)

[0.16666667 1.         0.25       0.25       0.125     ]
[0.16666667 1.         0.25       0.25       0.125     ]


다만 큰 array를 다룰 때의 속도는 매우 다릅니다.

In [85]:
%timeit (1.0 / big_array)

1.04 ms ± 10.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


이러한 벡터화된 계산은 *ufuncs*을 통해 이루어져 있습니다. 

또한 이러한 Ufuncs는 매우 유연합니다. Array와 Scalar를 모두 사용할 수 있습니다. 

In [86]:
np.arange(5) / np.arange(1, 6)

array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ])

또한 다차원 배열에도 사용 가능합니다.

In [87]:
x = np.arange(9).reshape((3, 3))
2 ** x

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]])

거의 모든 경우에 numpy array를 벡터화 하는 것이 python에서 loop를 쓰는 것 보다 빠릅니다. 특히 사이즈가 커지면 무조건이라고 보셔도 됩니다. 

즉, loop를 보시면 이것을 벡터화 할 수 있는지를 먼저 체크해보는 것이 좋습니다. 

## Exploring NumPy's UFuncs

Ufuncs는 두 가지 종류가 있습니다. 단일 입력을 받는 *unary ufuncs*, 두개의 입력을 받는 *binary ufuncs*가 있습니다. 

### Array arithmetic

사칙연산은 아래와 같습니다.

In [88]:
x = np.arange(4)
print("x     =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 2)
print("x / 2 =", x / 2)
print("x // 2 =", x // 2)  # floor division

x     = [0 1 2 3]
x + 5 = [5 6 7 8]
x - 5 = [-5 -4 -3 -2]
x * 2 = [0 2 4 6]
x / 2 = [0.  0.5 1.  1.5]
x // 2 = [0 0 1 1]


제곱을 위한 **, 나머지를 위한 % 역시 존재합니다. 

In [89]:
print("-x     = ", -x)
print("x ** 2 = ", x ** 2)
print("x % 2  = ", x % 2)

-x     =  [ 0 -1 -2 -3]
x ** 2 =  [0 1 4 9]
x % 2  =  [0 1 0 1]


더불어서 다양한 연산을 vector화 해서 사용할 수 있습니다. 

In [90]:
-(0.5*x + 1) ** 2

array([-1.  , -2.25, -4.  , -6.25])

사실 이 operation들은 binary ufuncs들의 wrapper입니다. 
예를 들어서 ``+`` 는 내부적으로는 ``np.add``를 호출합니다. 

In [91]:
np.add(x, 2)

array([2, 3, 4, 5])

아래를 참조하세요.

| Operator	    | Equivalent ufunc    | Description                           |
|---------------|---------------------|---------------------------------------|
|``+``          |``np.add``           |Addition (e.g., ``1 + 1 = 2``)         |
|``-``          |``np.subtract``      |Subtraction (e.g., ``3 - 2 = 1``)      |
|``-``          |``np.negative``      |Unary negation (e.g., ``-2``)          |
|``*``          |``np.multiply``      |Multiplication (e.g., ``2 * 3 = 6``)   |
|``/``          |``np.divide``        |Division (e.g., ``3 / 2 = 1.5``)       |
|``//``         |``np.floor_divide``  |Floor division (e.g., ``3 // 2 = 1``)  |
|``**``         |``np.power``         |Exponentiation (e.g., ``2 ** 3 = 8``)  |
|``%``          |``np.mod``           |Modulus/remainder (e.g., ``9 % 4 = 1``)|

### Absolute value

Numpy는 절대값도 Python의 기본 절대값 함수를 쓸 수 있긴 합니다. 

In [92]:
x = np.array([-2, -1, 0, 1, 2])
abs(x)

array([2, 1, 0, 1, 2])

대응되는 NumPy ufunc 으로 ``np.absolute`` 혹은 ``np.abs``가 있습니다. 두 함수는 같은 함수입니다.

In [93]:
np.absolute(x)

array([2, 1, 0, 1, 2])

In [94]:
np.abs(x)

array([2, 1, 0, 1, 2])

또한 Numpy는 복소수 계산도 가능합니다. 즉, *ufuncs*의 절대값은 복소수의 절대값을 반환합니다.

In [95]:
x = np.array([3 - 4j, 4 - 3j, 2 + 0j, 0 + 1j])
np.abs(x)

array([5., 5., 2., 1.])

### Trigonometric functions

Numpy는 심지어 삼각함수도 ufuncs로 내장해 줍니다. 데이터 분석에 꽤나 쓸만합니다. 

In [96]:
theta = np.linspace(0, np.pi, 3)

삼각함수는 아래와 같이 쓸 수 있습니다. 

In [97]:
print("theta      = ", theta)
print("sin(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))

theta      =  [0.         1.57079633 3.14159265]
sin(theta) =  [0.0000000e+00 1.0000000e+00 1.2246468e-16]
cos(theta) =  [ 1.000000e+00  6.123234e-17 -1.000000e+00]
tan(theta) =  [ 0.00000000e+00  1.63312394e+16 -1.22464680e-16]


컴퓨터의 Floating point error가 그대로 유지된다는 사실에 유의해 주세요.

당연히 역함수 또한 존재합니다:

In [98]:
x = [-1, 0, 1]
print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

x         =  [-1, 0, 1]
arcsin(x) =  [-1.57079633  0.          1.57079633]
arccos(x) =  [3.14159265 1.57079633 0.        ]
arctan(x) =  [-0.78539816  0.          0.78539816]


### Exponents and logarithms

로그함수와 지수함수도 지원합니다.

In [99]:
x = [1, 2, 3]
print("x     =", x)
print("e^x   =", np.exp(x))
print("2^x   =", np.exp2(x))
print("3^x   =", np.power(3, x))

x     = [1, 2, 3]
e^x   = [ 2.71828183  7.3890561  20.08553692]
2^x   = [2. 4. 8.]
3^x   = [ 3  9 27]


In [100]:
x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

x        = [1, 2, 4, 10]
ln(x)    = [0.         0.69314718 1.38629436 2.30258509]
log2(x)  = [0.         1.         2.         3.32192809]
log10(x) = [0.         0.30103    0.60205999 1.        ]


몇몇가지 특수한 경우를 위해 준비되어 있는 함수들도 있습니다. 만약 x가 매우 작다면, 아래의 함수들은 numerical precision에 의해서 np.log나 np.exp보다 조금 더 정확한 값을 줍니다. 

In [101]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

exp(x) - 1 = [0.         0.0010005  0.01005017 0.10517092]
log(1 + x) = [0.         0.0009995  0.00995033 0.09531018]


### Specialized ufuncs

여기서 다루지 않았지만 Numpy에는 훨씬 유용한 ufuncs들이 많습니다. 예를 들어서 hyperbolic 함수들, bit연산자, 비교연산자 등등이 있습니다. 수업 시간에 모두 다룰 수는 없고, numpy의 documentaion을 보면 자세히 나와 있습니다. 

또한 ``scipy.special``에는 numpy에 없지만 유용한 ``ufuncs``들을 제공해 줍니다. 

In [103]:
from scipy import special

In [104]:
# Gamma functions (generalized factorials) and related functions
x = [1, 5, 10]
print("gamma(x)     =", special.gamma(x))
print("ln|gamma(x)| =", special.gammaln(x))
print("beta(x, 2)   =", special.beta(x, 2))

gamma(x)     = [1.0000e+00 2.4000e+01 3.6288e+05]
ln|gamma(x)| = [ 0.          3.17805383 12.80182748]
beta(x, 2)   = [0.5        0.03333333 0.00909091]


In [105]:
# Error function (integral of Gaussian)
# its complement, and its inverse
x = np.array([0, 0.3, 0.7, 1.0])
print("erf(x)  =", special.erf(x))
print("erfc(x) =", special.erfc(x))
print("erfinv(x) =", special.erfinv(x))

erf(x)  = [0.         0.32862676 0.67780119 0.84270079]
erfc(x) = [1.         0.67137324 0.32219881 0.15729921]
erfinv(x) = [0.         0.27246271 0.73286908        inf]


NumPy와 ``scipy.special``의 수 많은 기능들을 수업에서 다루긴 힘듭니다만, 여러분이 확률 및 통계적 기법을 통해서 데이터를 다루고자 한다면 위의 두 가지의 *ufuncs*들의 설명문서를 찾아보시면 효율적인 계산법을 찾으실 수 있을겁니다.

## Advanced Ufunc Features

Ufuncs의 데이터를 조금 더 효율적으로 다루기 위해서 몇가지 기능을 더 소개해 드리겠습니다. 

### Specifying output

보통 아래와 같이 계산 결과를 특정 변수에 assign하게 되면, 임시 변수를 만들고 그 임시 변수의 데이터를 다시 변수에 복사해 주는 과정을 거칩니다

```python
x = np.arange(5)
y = np.multiply(x, 10)
```

이것을 바로 결과 array의 값을 치환하는 형태로 만들면 데이터의 크기와 종류에 따라서 더 효율적일 수 있습니다. 이 떄는 ``out``이라는 parameter를 사용합니다.

In [106]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y)
print(y)

[ 0. 10. 20. 30. 40.]


Array의 조회 방식을 통해서 특정 위치의 데이터만 바꾸는 것도 가능합니다.

예를 들어서 짝수번 index의 데이터로 계산 결과를 넣고 싶다면 아래 형태의 operation이 가능합니다.

In [107]:
y = np.zeros(10)
np.power(2, x, out=y[::2])
print(y)

[ 1.  0.  2.  0.  4.  0.  8.  0. 16.  0.]


### Aggregates

몇몇 binary ufuncs들은 한 array의 element를 그 성분으로 계산하게 할 수도 있습니다. 
이 때는``reduce``를 사용합니다. 

In [108]:
x = np.arange(1, 6)
print(np.add.reduce(x))
print(np.multiply.reduce(x))

15
120


또는 ``accumulate``를 통해서 값을 누적시킬 수 있습니다.

In [109]:
print(np.add.accumulate(x))
print(np.multiply.accumulate(x))

[ 1  3  6 10 15]
[  1   2   6  24 120]


이러한 Aggregation에 대해서는 다음시간에 조금 더 다루겠습니다.

### Outer products

벡터와 행렬의 외적을 구하는 것과 같은 방식으로, numpy의 array도 ``outer``를 통해 외적을 구할 수 있습니다.

In [110]:
x = np.arange(1, 6)
np.multiply.outer(x, x)

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])

## 이번 시간을 마치며
이번 시간에는 numpy의 기본적인 기능을 배워 보았습니다.

다음 시간에는 numpy의 중급(?) 기능들에 대해서 배워보겠습니다. 

다음 시간에 뵙겠습니다!